In [192]:
import pandas as pd

# TODO: use S3 to to fetch data from.
submission = pd.read_csv('./data/gender_submission.csv')
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')

print("\nTest Dataset")
print(train.drop(["Name"], axis=1).head())

print("\nColumns")
print(train.columns)

print("\nGet non empty col count")
print(train.count())
# print(test.count())


Test Dataset
   PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  \
0            1         0       3    male  22.0      1      0   
1            2         1       1  female  38.0      1      0   
2            3         1       3  female  26.0      0      0   
3            4         1       1  female  35.0      1      0   
4            5         0       3    male  35.0      0      0   

             Ticket     Fare Cabin Embarked  
0         A/5 21171   7.2500   NaN        S  
1          PC 17599  71.2833   C85        C  
2  STON/O2. 3101282   7.9250   NaN        S  
3            113803  53.1000  C123        S  
4            373450   8.0500   NaN        S  

Columns
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

Get non empty col count
PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch

## Analize the data

- First will not include `PassengerID`, `Name`, `Ticket`. Statistically does not makes any sence to me. 
- The `Cabin` is very important, but needs to think how to convert to nummeric. (Data is not presented in 25% cases)
- `Embarked` does not make sence to me atm.

`Survived` will be the Y value.

Lets now convert `Sex` to nummerical value and see train summary.

In [193]:
replace_gender = {'female': 1, "male": -1}  # all data given
test.Sex = test.Sex.replace(replace_gender)
train.Sex = train.Sex.replace(replace_gender)

/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipykernel_88590/2737735330.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test.Sex = test.Sex.replace(replace_gender)
/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipykernel_88590/2737735330.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train.Sex = train.Sex.replace(replace_gender)


### Cabin, Pclass, Fare, SibSp and Parch

Let see if Cabin, Pclass, Fare, SibSp and Parch has any relationship

In [194]:
train[["PassengerId", "Pclass", "Cabin", "Fare", "SibSp", "Parch", "Survived"]].dropna().head(15)

,PassengerId,Pclass,Cabin,Fare,SibSp,Parch,Survived
1,2,1,C85,71.2833,1,0,1
3,4,1,C123,53.1000,1,0,1
6,7,1,E46,51.8625,0,0,0
10,11,3,G6,16.7000,1,1,1
11,12,1,C103,26.5500,0,0,1
21,22,2,D56,13.0000,0,0,1
23,24,1,A6,35.5000,0,0,1
27,28,1,C23 C25 C27,263.0000,3,2,0
31,32,1,B78,146.5208,1,0,1
52,53,1,D33,76.7292,1,0,1


Indeed it has, passanger number 28 has high value of Fare, and it has 3 cabin values, additionally it has 3 family relations.
I would really better find the way how to represent cabin in numerical format, rather then using Fare. 
SibSp, Parch and Pclass will give me information about family members on the board and Cabin the vulnerability on the board, historically we know some blocks on the board were more vulnerable.

Lets try to convert Letter to digit and concatenate it with number

In [195]:

def calc_cabin(cabin):
    if pd.isna(cabin):
        return cabin
    cabins = cabin.split()
    for cabin in cabins:
        if len(cabins[0]) > 1:
            res = str(ord(cabins[0][0])) + cabins[0][1:]
            return res

train['Cabin'] = train['Cabin'].apply(calc_cabin)
test['Cabin'] = test['Cabin'].apply(calc_cabin)

In [196]:
print("\n Train Summary")
print(f"{test.describe()}")
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Cabin"]


 Train Summary
       PassengerId      Pclass         Sex         Age       SibSp  \
count   418.000000  418.000000  418.000000  332.000000  418.000000   
mean   1100.500000    2.265550   -0.272727   30.272590    0.447368   
std     120.810458    0.841838    0.963244   14.181209    0.896760   
min     892.000000    1.000000   -1.000000    0.170000    0.000000   
25%     996.250000    1.000000   -1.000000   21.000000    0.000000   
50%    1100.500000    3.000000   -1.000000   27.000000    0.000000   
75%    1204.750000    3.000000    1.000000   39.000000    1.000000   
max    1309.000000    3.000000    1.000000   76.000000    8.000000   

            Parch        Fare  
count  418.000000  417.000000  
mean     0.392344   35.627188  
std      0.981429   55.907576  
min      0.000000    0.000000  
25%      0.000000    7.895800  
50%      0.000000   14.454200  
75%      0.000000   31.500000  
max      9.000000  512.329200  


## DecisionTreeRegressor

In [197]:
from sklearn.tree import DecisionTreeRegressor


X = train[features]
y = train.Survived


model = DecisionTreeRegressor(random_state=0)
model.fit(X, y)


DecisionTreeRegressor(random_state=0)

In [198]:
print(f"On test data {model.predict(X.head(10))}")
print(f"Actual values {list(y.head(10))}")

On test data [0.         1.         1.         1.         0.         0.07894737
 0.         0.         1.         1.        ]
Actual values [0, 1, 1, 1, 0, 0, 0, 0, 1, 1]


In [199]:
from sklearn.metrics import mean_absolute_error

test_X = test[features]
test_y = submission.Survived


predicted = model.predict(test_X)
print(f"Predicted from test dataset {predicted[:10]}")
print(f"Actual val from test dataset {list(test_y[:10])}")

print("MAE", mean_absolute_error(predicted, test_y))

Predicted from test dataset [0.   0.   1.   0.75 0.   0.2  0.5  0.   0.5  1.  ]
Actual val from test dataset [0, 1, 0, 0, 1, 0, 1, 0, 1, 0]
MAE 0.24313835734762457


## Random Forest Tree

In [200]:
from sklearn.ensemble import RandomForestRegressor

model_forest = RandomForestRegressor(random_state=0)
model_forest.fit(X, y)

RandomForestRegressor(random_state=0)

In [201]:
forest_predict = model_forest.predict(test_X)

print("Random Forest predict: ", [round(i) for i in forest_predict[:10]])
print("Actual values: ", list(test_y[:10]))
print("MAE", mean_absolute_error(forest_predict, test_y))
print("MAE", mean_absolute_error([round(i) for i in forest_predict], test_y))

Random Forest predict:  [0, 0, 1, 1, 0, 0, 0, 0, 1, 0]
Actual values:  [0, 1, 0, 0, 1, 0, 1, 0, 1, 0]
MAE 0.22569131277768556
MAE 0.1507177033492823


In [202]:
res_df = pd.DataFrame({"P": [round(i) for i in forest_predict], "A": test_y})
# Rename the columns
res_df.columns = ['Prediction', 'Actual']

# Display the transposed DataFrame
res_df.to_csv('output.csv', index=False)


## 84.93% Accuracy